In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-3qkbe5sw/kobert-tokenizer_0c59e83099984a5d86279327bc2ea8be
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-3qkbe5sw/kobert-tokenizer_0c59e83099984a5d86279327bc2ea8be
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=fb7dc3cd85391d0a0eb1f36f14c668ee4dfaf6e21710196814f97a1469a21c9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-5f3dc9xi/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [4]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
# GPU 사용
device = torch.device("cuda:0")

In [6]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [13]:
import pandas as pd
chatbot_data1 = pd.read_csv('/content/drive/MyDrive/Final_project/4차년도.csv', encoding="CP949")
chatbot_data2 = pd.read_csv('/content/drive/MyDrive/Final_project/5차년도.csv', encoding="CP949")
chatbot_data3 = pd.read_csv('/content/drive/MyDrive/Final_project/5차년도_2차.csv', encoding="CP949")

df = pd.DataFrame()
df = pd.concat([pd.DataFrame(chatbot_data1['발화문']), pd.DataFrame(chatbot_data2['발화문']), pd.DataFrame(chatbot_data3['발화문'])], ignore_index=True)
df['상황'] = pd.concat([pd.DataFrame(chatbot_data1['상황']), pd.DataFrame(chatbot_data2['상황']), pd.DataFrame(chatbot_data3['상황'])], ignore_index=True)

df.loc[(df['상황'] == "fear"), '상황'] = 0  #fear => 0
df.loc[(df['상황'] == "surprise"), '상황'] = 1  #surprise => 1
df.loc[(df['상황'] == "anger"), '상황'] = 2  #anger => 2
df.loc[(df['상황'] == "angry"), '상황'] = 2  #angry => 2
df.loc[(df['상황'] == "sad"), '상황'] = 3  #sad => 3
df.loc[(df['상황'] == "sadness"), '상황'] = 3  #sadness => 3
df.loc[(df['상황'] == "neutral"), '상황'] = 4  #neutral => 4
df.loc[(df['상황'] == "happiness"), '상황'] = 5  #happiness => 5
df.loc[(df['상황'] == "disgust"), '상황'] = 6  #disgust => 6

data_list = []
for q, label in zip(df['발화문'], df['상황'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

In [14]:
emotion_dict = {0: 'fear', 1: 'surprise', 2: 'angry', 3: 'sad', 4: 'neutral', 5: 'happiness', 6: 'disgust'}

In [15]:
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=42)

In [16]:
print(len(dataset_train), '', len(dataset_test))

32993  10998


In [17]:
dataset_train[0:10]

[['나 축하해줘. 기분 좋은 일 있었어.', '5'],
 ['당연히 말씀 드려야지! 다시는 그렇게 행동 못 하게 혼쭐을 내야 해!', '6'],
 ['그거 괜찮은데?', '2'],
 ['지금 막 퇴근하고 들어왔는데 회사에서 또 전화왔어.', '2'],
 ['어제 우리 아빠가 또 다쳤어.', '2'],
 ['아! 짜증 나. 친구가 안 와.', '2'],
 ['전화는 안 할래.', '6'],
 ['프로젝트 하느라고 머리가 너무 아팠는데 이번 기회에 여행을 가서 머리 좀 식히고 와야겠어.', '5'],
 ['지금 친구한테 전화해서 약속 장소를 옮길지 약속을 취소할지 한 번 이야기 나누어봐야겠어. 괜히 만났다가 서로 아프게 되면 더 안 좋잖아.',
  '0'],
 ['다행히 해피는 안 다쳤어.', '1']]

In [25]:
class BERTSentenceTransform:
    
    def __init__(self, tokenizer, max_seq_length, vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab ##추가

    def __call__(self, line): 
        

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer(text_a)   
        #tokens_a = self._tokenizer.tokenize(text_a) 
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        #input_ids = tokens_a['input_ids']

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')



In [19]:
# Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [27]:
text_a = ' 나 축하해줘. 기분 좋은 일 있었어 .'
max_seq_length = max_len

tokens = []
tokens_a = tokenizer(text_a)
tokens.append(vocab.cls_token)
tokens.extend(tokens_a)
tokens.append(vocab.sep_token)
segment_ids = [0] * len(tokens)

input_ids = tokenizer.convert_tokens_to_ids(tokens)

valid_length = len(input_ids)

padding_length = max_seq_length - valid_length
input_ids.extend([vocab[vocab.padding_token]] * padding_length)
segment_ids.extend([0] * padding_length)

print('tokens : ', tokens)
print('segment_ids : ', segment_ids)
print('input_ids : ', input_ids)

tokens :  ['[CLS]', 'input_ids', 'token_type_ids', 'attention_mask', '[SEP]']
segment_ids :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
input_ids :  [2, 0, 0, 0, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'), np.array(segment_ids, dtype='int32')

(array([   2, 1370, 4566, 7848, 7303,   54, 1282, 4209, 3803, 3871, 6855,
         517,   54,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(14, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32))

In [9]:
text_a = '한국어 모델을 공유합니다.'

In [10]:
text_a = '한국어 모델을 공유합니다.'
tokens_1 = tokenizer.tokenize(text_a)
tokens_2 = tokenizer(text_a)
input_ids = tokenizer.convert_tokens_to_ids(tokens_1)
print(tokens_1, type(tokens_1))
print(tokens_2, type(tokens_2))
print(input_ids, type(input_ids))

['▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.'] <class 'list'>
{'input_ids': [2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]} <class 'transformers.tokenization_utils_base.BatchEncoding'>
[4958, 6855, 2046, 7088, 1050, 7843, 54] <class 'list'>


In [ ]:
tokens

['▁나', '▁축하', '해', '줘', '.', '▁기분', '▁좋은', '▁일', '▁있었', '어', '▁', '.']

In [ ]:
input_ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids

[1370, 4566, 7848, 7303, 54, 1282, 4209, 3803, 3871, 6855, 517, 54]

In [20]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))



In [ ]:
dataset_train_test = [['나 축하해줘. 기분 좋은 일 있었어.', '5'],
 ['당연히 말씀 드려야지! 다시는 그렇게 행동 못 하게 혼쭐을 내야 해!', '6'],
 ['그거 괜찮은데?', '2']]

In [24]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [ ]:
type(data_train[0][0])

numpy.ndarray

In [ ]:
len(data_train)

32993

In [ ]:
for i in range(10, 15):
  print(data_train[i])

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
train_dataloader

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 2/1032 [00:01<10:51,  1.58it/s]

epoch 1 batch id 1 loss 2.122025489807129 train acc 0.0625


 20%|█▉        | 202/1032 [00:25<01:36,  8.57it/s]

epoch 1 batch id 201 loss 1.4375696182250977 train acc 0.32431592039800994


 39%|███▉      | 402/1032 [00:48<01:14,  8.40it/s]

...

epoch 5 batch id 1001 loss 0.07382971793413162 train acc 0.9778034465534465


100%|██████████| 1032/1032 [02:02<00:00,  8.44it/s]


epoch 5 train acc 0.9779554263565892


100%|██████████| 344/344 [00:12<00:00, 26.83it/s]

epoch 5 test acc 0.9278706395348837


In [ ]:
torch.save(model, '/content/drive/MyDrive/Final_project/model.pt')

In [ ]:
model = torch.load("/content/drive/MyDrive/Final_project/model.pt", map_location=device)
print(model)

In [ ]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
    for tensor in [token_ids, valid_length, segment_ids, label]:
        if not tensor.is_contiguous():
            print("Found non-contiguous tensor:", tensor.shape)


In [ ]:
#토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]
    emotion_dict = {0: 'fear', 1: 'surprise', 2: 'angry', 3: 'sad', 4: 'neutral', 5: 'happiness', 6: 'disgust'}
    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)

    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


#         test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            emotion = emotion_dict[np.argmax(logits)]
            
        print(out)
        print(f">> 입력하신 내용의 감정은 {emotion}입니다.")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
import torch.nn.functional as F

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    print('max_vals: ', max_vals, ' ,max_indices : ', max_indices)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    print('train_acc : ', train_acc)
    return train_acc

def predict_test(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]
    emotion_dict = {0: 'fear', 1: 'surprise', 2: 'angry', 3: 'sad', 4: 'neutral', 5: 'happiness', 6: 'disgust'}
    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)

    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        print('batch_id : ', batch_id, ' ,(token_ids, valid_length, segment_ids, label) : ', (token_ids, valid_length, segment_ids, label))
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        print('out : ', out, '  ,type : ', type(out), '\n')
        prob = F.softmax(out, dim=0)
        print('prob : ' ,prob)
        calc_accuracy(out, label)

#         test_eval=[]
        for i in out:
          print('i : ', i, '  ,type : ', type(i), '\n')
          logits=i
          logits = logits.detach().cpu().numpy()
          print('logits :', logits, '  ,type : ', type(logits), '\n')
          print('np.argmax(logits) : ', np.argmax(logits), '  ,type : ', type(np.argmax(logits)), '\n')
          emotion = emotion_dict[np.argmax(logits)]
            
        
        print(f">> 입력하신 내용의 감정은 {emotion}입니다.")

In [ ]:
a = [-1.2185, -2.1005, -1.7756,  1.2294,  6.0908, -1.1891, -0.8650]
sum(a)

0.17149999999999999

In [ ]:
predict_test('내 감정이 어떤지 잘 모르겠어 .')

batch_id :  0  ,(token_ids, valid_length, segment_ids, label) :  (tensor([[   2, 1434,  803, 7096, 3224, 7318, 3942, 2049, 5405, 6855,  517,   54,
            3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1]], dtype=torch.int32), tensor([13], dtype=torch.int32), tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32), tensor([0], dtype=torch.int32))
out :  tensor([[-1.8566, -2.3292,  0.4924,  7.3638, -1.0001, -2.0952, -1.2824]],
       device='cuda:0', grad_fn=<AddmmBackward0>)   ,type :  <class 'torch.Tensor'> 

p

In [ ]:
import torch.nn.functional as F

out = tensor([-1.2185, -2.1005, -1.7756, 1.2294, 6.0908, -1.1891, -0.8650])
prob = F.softmax(out, dim=0)
prob

NameError: ignored

In [ ]:
while True:
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence.upper() == "QUIT":
        print("감정 분석을 종료합니다.")
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 나 화나
tensor([[-2.7542,  0.0747,  2.8325, -1.8391, -0.1198, -1.6196,  2.9361]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
>> 입력하신 내용의 감정은 disgust입니다.


하고싶은 말을 입력해주세요 : 너 정말 역겨워 죽겠어
tensor([[-1.3468, -0.9651, -0.7582, -1.8387, -1.0271, -1.9761,  7.4183]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
>> 입력하신 내용의 감정은 disgust입니다.


하고싶은 말을 입력해주세요 : QUIT
감정 분석을 종료합니다.
